In [1]:
from stonks.data.data_reader import MktDataReader
from stonks.models.market_models import MarketModels
from stonks.models.asset_allocation import AssetAllocation


from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions 

In [2]:
tickers = [
    'AAPL', # Apple -> tech
    'TSLA', # Tesla -> tech
    'GOOGL', # Google -> tech
    'AMZN', # Amazon -> consumer/tech
    'MA', # MasterCard -> finance
    'V', # Visa -> finance
    'MELI', # MercadoLibre -> consumer
    'NVDA', # Nvidia -> chips
    'ASML', # ASML -> chips
    'FB' # facebook -> social media
]

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-04-05'

dr = MktDataReader(start_date, end_date, 'yahoo', tickers)

[*********************100%***********************]  10 of 10 completed


In [3]:
# impute missing data
dr.impute_missing_data()
# fetch market caps
dr.fetch_market_caps()

In [4]:
## Fama-French

In [5]:
dr.ff_factors.head(2)

,Mkt-RF,SMB,HML,RMW,CMA,RF
1963-07-01,-0.67,0.00,-0.34,-0.01,0.15,0.012
1963-07-02,0.79,-0.27,0.27,-0.07,-0.19,0.012


In [6]:
ff_model = MarketModels(
    dr.df_stocks_bkfilled,  # stocks historical data from the MktDataReader class
    model = "vanilla-ff",  # name of the model to get the estimated returns + covar matrices
    mcaps = dr.mcaps,  # market caps from the MktDataReader class
    ff_factors_df = dr.ff_factors  # fetched fama-french factors
)

[*********************100%***********************]  1 of 1 completed


In [7]:
ff_model.er_fama_df.head(2)

,AAPL,AMZN,ASML,FB,GOOGL,MA,MELI,NVDA,TSLA,V
2008-01-02,-0.001657,-0.000989,-0.007272,0.006215,-0.000627,-0.004029,-0.007214,-0.009185,-0.000267,-0.001893
2008-01-03,0.013003,0.015102,0.008010,0.014031,0.013892,0.011836,0.011479,0.010585,0.009397,0.011685


In [8]:
dict(ff_model.ret_ff)

{'AAPL': 0.6893613362394948,
 'AMZN': 0.6829168052582064,
 'ASML': 0.7013237128562366,
 'FB': 0.6140032479618611,
 'GOOGL': 0.6623349283413695,
 'MA': 0.6726381123578156,
 'MELI': 0.7146575977165455,
 'NVDA': 0.732607865139415,
 'TSLA': 0.6451419499963542,
 'V': 0.6531904851041315}

In [9]:
ff_model.ff_scores  # R^2 scores used as a proxy for confidences

{'AAPL': 0.48643685745467335,
 'AMZN': 0.4288670306553818,
 'ASML': 0.4766886582967995,
 'FB': 0.17717704139763246,
 'GOOGL': 0.5179378591487357,
 'MA': 0.5316609922170439,
 'MELI': 0.37135253977967697,
 'NVDA': 0.42787103193722387,
 'TSLA': 0.16524165013475411,
 'V': 0.4830145933957265}

In [10]:
## Black-Litterman with FF factors
# views taken from the FF-model
viewdict = dict(ff_model.ret_ff)
dict_keys = [k for k in viewdict.keys()]

# confidences
confidences = [ff_model.ff_scores[k] for k in dict_keys]

# initiate the stonks MarketModels obj
bl_ff_mod = MarketModels(
    dr.df_stocks_bkfilled,  # stocks historical data from the MktDataReader class
    model = "bl",  # name of the model to get the estimated returns + covar matrices
    views_dict = viewdict,  # own views on expectations of stocks
    confidences = confidences,  # how sure are you about your views of the market
    mcaps = dr.mcaps  # market caps from the MktDataReader class
)

[*********************100%***********************]  1 of 1 completed


In [12]:
bl_ff_mod.market_prior

AAPL     0.198871
AMZN     0.216147
ASML     0.180064
FB       0.125017
GOOGL    0.183439
MA       0.175630
MELI     0.227645
NVDA     0.234007
TSLA     0.179788
V        0.156072
dtype: float64

In [14]:
print(bl_ff_mod.ret_bl)
print(type(bl_ff_mod.S_bl))

AAPL     0.577333
AMZN     0.607355
ASML     0.584925
FB       0.349906
GOOGL    0.555431
MA       0.593431
MELI     0.699569
NVDA     0.702099
TSLA     0.463361
V        0.532468
dtype: float64
<class 'pandas.core.frame.DataFrame'>


In [15]:
# using the computed black-litterman model
al = AssetAllocation(bl_ff_mod.ret_bl, bl_ff_mod.S_bl, "markowitz-regularized")
al.portfolio_perf

Expected annual return: 58.1%
Annual volatility: 26.5%
Sharpe Ratio: 2.11


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.5805289924765453, 0.2653722667296166, 2.1122365173436113)

In [16]:
# portfolio weights
al.weights

OrderedDict([('AAPL', 0.10975),
             ('AMZN', 0.11093),
             ('ASML', 0.10291),
             ('FB', 0.06559),
             ('GOOGL', 0.10594),
             ('MA', 0.10962),
             ('MELI', 0.1068),
             ('NVDA', 0.11517),
             ('TSLA', 0.0737),
             ('V', 0.0996)])

In [17]:
## Black-Litterman

In [18]:
# declare our views of the market
viewdict = {
    "GOOGL": 0.30,
    "FB": 0.30,
    "ASML": 0.30,
    "NVDA": 0.30,
}

# how confident we are on our views
confidences = [
    0.8,
    0.8,
    0.6,
    0.6,
]

# initiate the stonks MarketModels obj
bl = MarketModels(
    dr.df_stocks_bkfilled,  # stocks historical data from the MktDataReader class
    model = "bl",  # name of the model to get the estimated returns + covar matrices
    views_dict = viewdict,  # own views on expectations of stocks
    confidences = confidences,  # how sure are you about your views of the market
    mcaps = dr.mcaps  # market caps from the MktDataReader class
) 

[*********************100%***********************]  1 of 1 completed


In [19]:
# which model we used + the trained model obj
bl.model_summary

{'name': 'Black-litterman',
 'model': <pypfopt.black_litterman.BlackLittermanModel at 0x7fa8aa25be10>}

In [20]:
print(type(bl.ret_bl))
print(type(bl.S_bl))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [21]:
bl.ret_bl

AAPL     0.276361
AMZN     0.307078
ASML     0.274076
FB       0.269517
GOOGL    0.285990
MA       0.252113
MELI     0.330368
NVDA     0.321420
TSLA     0.255788
V        0.224633
dtype: float64

In [22]:
# asset allocation

In [23]:
# using the computed black-litterman model
al = AssetAllocation(bl.ret_bl, bl.S_bl, "markowitz-regularized")
al.portfolio_perf

Expected annual return: 28.2%
Annual volatility: 25.8%
Sharpe Ratio: 1.01


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.28221668383820614, 0.2583957205268486, 1.0147872546169374)

In [24]:
# portfolio weights
al.weights

OrderedDict([('AAPL', 0.10436),
             ('AMZN', 0.11523),
             ('ASML', 0.09511),
             ('FB', 0.12087),
             ('GOOGL', 0.11327),
             ('MA', 0.0856),
             ('MELI', 0.10024),
             ('NVDA', 0.1031),
             ('TSLA', 0.08596),
             ('V', 0.07626)])